# Introduction to Bingo
Bingo is an open source package for performing symbolic regression, Though it 
can be used as a general purpose evolutionary optimization package.  

### Key Features
  * Integrated local optimization strategies
  * Parallel island evolution strategy implemented with mpi4py
  * Coevolution of fitness predictors

In [28]:
import numpy as np
from bingo.Base.MultipleValues import SinglePointCrossover
from bingo.Base.MultipleValues import SinglePointMutation

def get_random_float():
    return np.random.random_sample()

crossover = SinglePointCrossover()
mutation = SinglePointMutation(get_random_float)

In [29]:
from bingo.Base.FitnessFunction import FitnessFunction
from bingo.Base.ContinuousLocalOptimization import ContinuousLocalOptimization
from bingo.Base.Evaluation import Evaluation

class ZeroMinFitnessFunction(FitnessFunction):
    def __call__(self, individual):
        return np.linalg.norm(individual.values)

                              
fitness = ZeroMinFitnessFunction()
local_opt_fitness = ContinuousLocalOptimization(fitness)
evaluator = Evaluation(local_opt_fitness)

In [30]:
from bingo.Base.TournamentSelection import Tournament

GOAL_POPULATION_SIZE = 25

selection = Tournament(GOAL_POPULATION_SIZE)

In [31]:
from bingo.Base.MuPlusLambdaEA import MuPlusLambda

MUTATION_PROBABILITY = 0.4
CROSSOVER_PROBABILITY = 0.4
NUM_OFFSPRING = GOAL_POPULATION_SIZE

evo_alg = MuPlusLambda(evaluator,
                       selection,
                       crossover,
                       mutation,
                       CROSSOVER_PROBABILITY,
                       MUTATION_PROBABILITY,
                       NUM_OFFSPRING)

In [32]:
from bingo.Base.MultipleFloats import MultipleFloatChromosomeGenerator

VALUE_LIST_SIZE = 8

generator = MultipleFloatChromosomeGenerator(get_random_float, VALUE_LIST_SIZE)

In [33]:
from bingo.Base.Island import Island

POPULATION_SIZE = 25

island = Island(evo_alg, generator, POPULATION_SIZE)

In [34]:
def print_population(island):
    for i, indv in enumerate(island.population):
            print("indv", i, ["{0:.2f}".format(val) for val in indv.values])

In [35]:
best_indv_values = []
best_indv_values.append(island.best_individual().values)
for i in range(500):
    island.execute_generational_step()
    best_indv_values.append(island.best_individual().values)

In [36]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

In [37]:
from IPython.display import HTML

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def animate_data(list_of_best_indv_values):

    fig, ax = plt.subplots()

    num_generations = len(list_of_best_indv_values)
    x = np.arange(0, len(list_of_best_indv_values[0]))
    y = list_of_best_indv_values
    zero = [0]*len(x)
    polygon = ax.fill_between(x, zero, y[0], color='b', alpha=0.3)
    points, = ax.plot(x, y[0], 'bs')
    points.set_label('Generation :' + str(0))
    legend = ax.legend(loc='upper right', shadow=True)


    def animate(i):
        ax.collections.clear()
        polygon = ax.fill_between(x, zero, y[i], color='b', alpha=0.3)
        points.set_ydata(y[i])  # update the data
        points.set_label('Generation :' + str(i))
        legend = ax.legend(loc='upper right')
        return points, polygon, legend


    # Init only required for blitting to give a clean slate.
    def init():
        points.set_ydata(np.ma.array(x, mask=True))
        return points, polygon, points

    plt.xlabel('Chromosome Value Index', fontsize=15)
    plt.ylabel('Value Magnitude', fontsize=15)
    plt.title("Values of Best Individual in Island", fontsize=15)
    plt.ylim(-0.01,2.05)
    ax.tick_params(axis='y', labelsize=15)
    ax.tick_params(axis='x', labelsize=15)

    ani = animation.FuncAnimation(fig, animate, num_generations, init_func=init,
                                interval=250, blit=True)
    return ani

In [43]:
anim = animate_data(best_indv_values)
display_animation(anim)

MovieWriter ffmpeg unavailable. Trying to use pillow instead.


ValueError: unknown file extension: .mp4

In [ ]:
print_population(island)

In [ ]:
from bingo.SymbolicRegression.ExplicitRegression import ExplicitTrainingData

NUM_POINTS = 100
START = -10
STOP = 10

x = np.linspace(START, STOP, NUM_POINTS).reshape([-1, 1])
y = x**2 + 3.5*x**3
training_data = ExplicitTrainingData(x, y)

In [ ]:
from bingo.SymbolicRegression.AGraph.ComponentGenerator import ComponentGenerator

ADD = 2
SUB = 3
MULT = 4

component_generator = ComponentGenerator(x.shape[1])
component_generator.add_operator(ADD)
component_generator.add_operator(SUB)
component_generator.add_operator(MULT)

In [ ]:
from bingo.SymbolicRegression.AGraph.AGraphCrossover import AGraphCrossover
from bingo.SymbolicRegression.AGraph.AGraphMutation import AGraphMutation

crossover = AGraphCrossover()
mutation = AGraphMutation(component_generator)

In [ ]:
from bingo.SymbolicRegression.ExplicitRegression import ExplicitRegression

fitness = ExplicitRegression(training_data=training_data)
local_opt_fitness = ContinuousLocalOptimization(fitness, algorithm='lm')
evaluator = Evaluation(local_opt_fitness)

In [ ]:
from bingo.SymbolicRegression.AGraph.AGraphGenerator import AGraphGenerator

POP_SIZE = 128
STACK_SIZE = 10
ERROR_TOLERANCE = 1e-6

agraph_generator = AGraphGenerator(STACK_SIZE, component_generator)

In [ ]:
from bingo.Base.AgeFitnessEA import AgeFitnessEA

ea = AgeFitnessEA(evaluator, agraph_generator, crossover,
                      mutation, MUTATION_PROBABILITY,
                      CROSSOVER_PROBABILITY, POP_SIZE)

island = Island(ea, agraph_generator, POP_SIZE)

In [ ]:
i = 1
while island.best_individual().fitness > ERROR_TOLERANCE:
    island.execute_generational_step()
    i += 1

print("Generation: ", i)
print("Success!", island.best_individual().get_latex_string())